# SHC / TC / TD Prediction - ThermoProfiler
*Predicting Petrophysical Properties from Well Logs*

- Upload your well log data 
- Select **rock type, model, and target variable**
- Run predictions using pre-trained models


In [1]:
# MAKE SURE OF CORRECT VERSIONS
#pip install scikit-learn==1.2.1
#pip install numpy==1.25.2

import pandas as pd
import numpy as np
from joblib import load
import sys
import os
sys.path.append(os.path.abspath("..")) 

sys.path.append(os.path.abspath("../thermoprofiler")) 

### STEP 1 -  LOAD DATA

In [2]:
df = pd.read_csv(
    r"C:\Users\dergun\Documents\SHC-TC-TD-Prediction-using-petrophysical-well-logs\notebook\Example_well_input (1).txt",
    encoding="utf-16",   
    sep="\t", 
    header=0
)

# Drop first row (often metadata) and reset index
df = df.drop(0).reset_index(drop=True)
df

,Depth,Stratigraphy,Temp,GR,CAL,PHIN,RHOB,VSH,Lithology,Rock_type
0,0.00,Quaternary-Saale,-999.25,8.59859,-999,-999,-999,-999.25,S,3
1,0.25,Quaternary-Saale,-999.25,8.59859,-999,-999,-999,0.065429,S,3
2,0.50,Quaternary-Saale,-999.25,7.65106,-999,-999,-999,0.0482012,S,3
3,0.75,Quaternary-Saale,-999.25,7.70233,-999,-999,-999,0.0491333,S,3
4,1.00,Quaternary-Saale,-999.25,9.43716,-999,-999,-999,0.0806757,S,3
...,...,...,...,...,...,...,...,...,...,...
4396,1099.00,nodata,-999.25,-999.25,-999,-999,-999,-999.25,nodata,nodata
4397,1099.25,nodata,-999.25,-999.25,-999,-999,-999,-999.25,nodata,nodata
4398,1099.50,nodata,-999.25,-999.25,-999,-999,-999,-999.25,nodata,nodata
4399,1099.75,nodata,-999.25,-999.25,-999,-999,-999,-999.25,nodata,nodata


### STEP 2 - STANDARIZE COLUMNS AND CLEAN COLUMS


### STEP 4 - ENSURE CORRECT UNITS FOR PHIN AND VSH ( % -> FRACTIONS)

In [ ]:
from thermoprofiler.model_selector import assign_model_numbers
from thermoprofiler.preprocessing import clean_log_dataframe
from thermoprofiler.prediction import predict_all_properties


df = clean_log_dataframe(df)          
df = assign_model_numbers(df)       


df = predict_all_properties(df, model_type="ADABOOST")

df.head()
df.value_counts()
df.tail(500)

Trying to load model at path: c:\Users\dergun\Documents\thermoprofiler\compiled_models/Clastics/XGBoost/TC/1.joblib
Exists: True
Full path: c:\Users\dergun\Documents\thermoprofiler\compiled_models\Clastics\XGBoost\TC\1.joblib


c:\Users\dergun\miniforge3\envs\heatflow_work\Lib\pickle.py:1718: UserWarning: [18:17:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


Trying to load model at path: c:\Users\dergun\Documents\thermoprofiler\compiled_models/Clastics/XGBoost/SHC/1.joblib
Exists: True
Full path: c:\Users\dergun\Documents\thermoprofiler\compiled_models\Clastics\XGBoost\SHC\1.joblib
Trying to load model at path: c:\Users\dergun\Documents\thermoprofiler\compiled_models/Clastics/XGBoost/TD/1.joblib
Exists: True
Full path: c:\Users\dergun\Documents\thermoprofiler\compiled_models\Clastics\XGBoost\TD\1.joblib
Trying to load model at path: c:\Users\dergun\Documents\thermoprofiler\compiled_models/Clastics/XGBoost/TC/3.joblib
Exists: True
Full path: c:\Users\dergun\Documents\thermoprofiler\compiled_models\Clastics\XGBoost\TC\3.joblib
Trying to load model at path: c:\Users\dergun\Documents\thermoprofiler\compiled_models/Clastics/XGBoost/SHC/3.joblib
Exists: True
Full path: c:\Users\dergun\Documents\thermoprofiler\compiled_models\Clastics\XGBoost\SHC\3.joblib
Trying to load model at path: c:\Users\dergun\Documents\thermoprofiler\compiled_models/Clast

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\dergun\\Documents\\thermoprofiler\\compiled_models/Clastics/XGBoost/TC/5.joblib'

### STEP 5 - ASSIGN MODEL NUMBER PER INTERVALL
Assigns a model number to each row in the dataset depending on which combination of logs is available in the row.
| Model Number | Logs Included                  |
| -- | ------------------------------ |
| 1  | \['RHOB']                      |
| 2  | \['PHIN']                      |
| 3  | \['VSH']                       |
| 4  | \['Vp']                        |
| 5  | \['RHOB', 'PHIN']              |
| 6  | \['RHOB', 'VSH']               |
| 7  | \['RHOB', 'Vp']                |
| 8  | \['PHIN', 'VSH']               |
| 9  | \['PHIN', 'Vp']                |
| 10 | \['VSH', 'Vp']                 |
| 11 | \['RHOB', 'PHIN', 'VSH']       |
| 12 | \['RHOB', 'PHIN', 'Vp']        |
| 13 | \['RHOB', 'VSH', 'Vp']         |
| 14 | \['PHIN', 'VSH', 'Vp']         |
| 15 | \['RHOB', 'PHIN', 'VSH', 'Vp'] |


### STEP 6 - PREDICT THERMAL PROPERTIES 
In this step, thermal properties (e.g., thermal conductivity (TC) and heat capacity (SHC)) are predicted. 
Rock type options:

- Carbonates
-  Evaporites
-   Clastics

Model options:

-   Linear Regression
-   AdaBoost
-   XGBoost
-   Random Forest (RF)

Target properties:
-   Thermal Difussivity (TD)
-   Thermal Conductivity (TC)
-   Heat Capacity (SHC)